# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

In [2]:
def column_pivoting(a, n):
    """
    a --> a square matrix
    n --> number of pivot which we may change
    """
    N = a.shape[0]
    
    arg_max = n
    max = a[n,n]
    
    for i in range(n, N):
        if abs(a[n, i]) > max:
            arg_max = i
            max = abs(a[n, i])
    
    p = np.eye(N)
    if arg_max != n:
        p[n,n], p[i,i] = 0, 0
        p[n,i], p[i,n] = 1, 1
        a[:,[n,i]] = a[:,[i,n]] # Cheapest way to swap two columns
    return p, a    
    

In [3]:
def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    With pivoting.
    A = LU(P.T)
    for permuation matrix which P is, the inverse of matrix is equal to transpose of that matrix.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):
        p, u = column_pivoting(u, j)
        P = P @ p
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u, P.T

In [4]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [5]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [6]:
L, u, _ = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.65  1.    0.    0.   ]
 [1.    1.231 2.09  2.01  1.    0.   ]
 [1.    1.25  2.406 2.875 2.346 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -2.475e-01 -1.437e-01 -6.259e-02]
 [ 0.000e+00  2.220e-16  0.000e+00  2.421e-02  2.021e-02  1.030e-02]
 [ 0.000e+00 -2.242e-16  0.000e+00  0.000e+00 -6.462e-04 -5.096e-04]
 [ 0.000e+00 -1.124e-16  0.000e+00 -1.238e-18  0.000e+00  6.730e-06]] 

[[ 0.     0.     0.     0.     0.     0.   ]
 [ 0.    -1.125  0.511  0.292  0.189  0.132]
 [ 0.    -0.935  0.481  0.23   0.135  0.089]
 [ 0.    -0.771  0.419  0.183  0.103  0.066]
 [ 0.    -0.652  0.365  0.151  0.083  0.052]
 [ 0.    -0.562  0.321  0.129  0.069  0.043]]


# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [7]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [8]:
np.linalg.matrix_rank(a1)

6

In [9]:
print(a1)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


In [10]:
l, u, _ = diy_lu(a1)

print(l, u)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.179 1.    0.    0.   ]
 [1.    1.231 1.294 1.605 1.    0.   ]
 [1.    1.25  1.375 2.007 2.076 1.   ]] [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -2.475e-01 -1.437e-01 -6.259e-02]
 [ 0.000e+00  2.220e-16  2.220e-16 -9.247e-02 -4.755e-02 -1.920e-02]
 [ 0.000e+00 -1.344e-16 -3.565e-16  0.000e+00  1.841e-03  1.181e-03]
 [ 0.000e+00 -1.667e-16  2.941e-16 -1.567e-17 -4.337e-19 -1.233e-05]]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [11]:
# ... ENTER YOUR CODE HERE ...
def leading_minors_is_non_zero(a):
    N = a.shape[0]
    answer = True
    for i in range(N):
        a_ixi = a[i,i]
        for j in range(i):
            a_ixi -= a[i,j] / a[j,j] * a[j,i]
        if(1 + a_ixi == 1): # because we may have a number smaller than machine epsilon instead of zero that we should have!
            answer = False
    return answer

In [12]:
print(leading_minors_is_non_zero(a))
print(leading_minors_is_non_zero(a1))

True
False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [13]:
# ... ENTER YOUR CODE HERE ...
def reconstruct(l, u, p):
    """
    return matrix A which is original matrix
    """
    return l@u@p

In [14]:
l, u, p = diy_lu(a)
a_recons = reconstruct(l, u, p)
print("matrix a before decomposition")
print(a)
print("matrix a after reconstruction")
print(a_recons)
l, u, p = diy_lu(a1)
a1_recons = reconstruct(l, u, p)
print("matrix a1 before decomposition")
print(a1)
print("matrix a1 after reconstruction")
print(a1_recons)

matrix a before decomposition
[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
matrix a after reconstruction
[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
matrix a1 before decomposition
[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
matrix a1 after reconstruction
[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.5